In [1]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen

import pandas as pd
import re

In [2]:
df = pd.read_csv('data/chicagomag_info.csv', index_col='Rank') 
df.head()

,Unnamed: 0,Cafe,Menu,URL
Rank,,,,
1,0,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,1,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,2,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,3,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,4,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [3]:
del df['Unnamed: 0']
df.head()

,Cafe,Menu,URL
Rank,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 50
Data columns (total 3 columns):
Cafe    50 non-null object
Menu    50 non-null object
URL     50 non-null object
dtypes: object(3)
memory usage: 1.6+ KB


# 첫번째 URL 페이지 크롤링하기

In [5]:
html = urlopen(df['URL'][1])
soup = BeautifulSoup(html, "lxml")
soup.title
# type(soup)

<title>
  1. Old Oak Tap BLT |
  Chicago magazine
      |  November 2012
    </title>

In [6]:
soup.find('p', 'addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [7]:
taginfo = soup.find('p', 'addy').get_text()
taginfo

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [8]:
taginfo = taginfo[taginfo.find('$'):]
taginfo

'$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

#Regular Expression

In [9]:
price = re.search('\$\d+\.(\d+)?', taginfo).group()
price

'$10.'

In [13]:
address = '일단보류'

In [14]:
phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
if phone is not None:
    phone = phone.group()
phone

'773-772-0406'

In [15]:
infolist = taginfo.split(', ')
if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
    homepage = infolist[-1]
else:
    homepage = 'No Homepage'
homepage

'theoldoaktap.com'

In [16]:
start = len(price)
if phone is None:
    phone = 'No Contact'
    end = taginfo.find(homepage)
else:
    end = taginfo.find(phone)
    
address = taginfo[start:end]
address

' 2109 W. Chicago Ave., '

In [17]:
#상위 TOP5 페이지에서 가격과 주소정보만 읽어오기

In [18]:
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in [1,2,3,4,5]:   
    html = urlopen(df['URL'][n])
    soup = BeautifulSoup(html, 'lxml')
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 

    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print(n)

1
2
3
4
5


In [19]:
Price

['$10.', '$9.', '$9.50', '$9.40', '$10.']

In [20]:
price = Price[0]
if price.endswith('.'):
    price = price[:-1]
price

'$10'

In [21]:
Address

[' 2109 W. Chicago Ave., ',
 ' 800 W. Randolph St., ',
 '. 445 N. Clark St., ',
 '. 914 Noyes St., Evanston, ',
 ' 825 W. Fulton Mkt., ']

In [22]:
address = Address[2]
address = address[:-2]
if address.startswith('.'):
    address = address[1:]
address = address.strip()
address

'445 N. Clark St.'

In [23]:
#가격과 주소정보를 수정보완 

In [24]:
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in [1,2,3,4,5]:   
    html = urlopen(df['URL'][n])
    soup = BeautifulSoup(html, 'lxml')
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    # price update
    if price.endswith('.'):
        price = price[:-1]
        
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 
    # address update
    address = address[:-2]
    if address.startswith('.'):
        address = address[1:]
    address = address.strip()
    
    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print(n)

1
2
3
4
5


In [25]:
Price

['$10', '$9', '$9.50', '$9.40', '$10']

In [26]:
Address

['2109 W. Chicago Ave.',
 '800 W. Randolph St.',
 '445 N. Clark St.',
 '914 Noyes St., Evanston',
 '825 W. Fulton Mkt.']

In [27]:
Price    = []
Address  = []
Phone    = []
Homepage = []

for n in df.index:
    html = urlopen(df['URL'][n])
    soup = BeautifulSoup(html, 'lxml')
    
    taginfo = soup.find('p', 'addy').get_text()
    taginfo = taginfo[taginfo.find('$'):]
    
    price = re.search('\$\d+\.(\d+)?', taginfo).group()
    # price update
    if price.endswith('.'):
        price = price[:-1]
        
    address = '일단보류'    # Street, City
    
    phone = re.search('\d{3}[-]\d{3}[-]\d{4}', taginfo)
    if phone is not None:
        phone = phone.group()    
        
    infolist = taginfo.split(', ')
    if infolist[-1].endswith('.com') or infolist[-1].endswith('.net'):
        homepage = infolist[-1]
    else:
        homepage = 'No Homepage'

    start = len(price)
    if phone is None:
        phone = 'No Contact'
        end = taginfo.find(homepage)
    else:
        end = taginfo.find(phone)

    address = taginfo[start:end] 
    # address update
    address = address[:-2]
    if address.startswith('.'):
        address = address[1:]
    address = address.strip()
    
    Price.append(price)
    Address.append(address)
    Phone.append(phone)
    Homepage.append(homepage)
    
    print('webpage crawling...{}'.format(n))

webpage crawling...1
webpage crawling...2
webpage crawling...3
webpage crawling...4
webpage crawling...5
webpage crawling...6
webpage crawling...7
webpage crawling...8
webpage crawling...9
webpage crawling...10
webpage crawling...11
webpage crawling...12
webpage crawling...13
webpage crawling...14
webpage crawling...15
webpage crawling...16
webpage crawling...17
webpage crawling...18
webpage crawling...19
webpage crawling...20
webpage crawling...21
webpage crawling...22
webpage crawling...23
webpage crawling...24
webpage crawling...25
webpage crawling...26
webpage crawling...27
webpage crawling...28
webpage crawling...29
webpage crawling...30
webpage crawling...31
webpage crawling...32
webpage crawling...33
webpage crawling...34
webpage crawling...35
webpage crawling...36
webpage crawling...37
webpage crawling...38
webpage crawling...39
webpage crawling...40
webpage crawling...41
webpage crawling...42
webpage crawling...43
webpage crawling...44
webpage crawling...45
webpage crawling...

In [28]:
len(df), len(Price), len(Address), len(Phone), len(Homepage)

(50, 50, 50, 50, 50)

In [29]:
df['Price'] = Price
df.head()

,Cafe,Menu,URL,Price
Rank,,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10


In [30]:
df['Address'] = Address
df.head()

,Cafe,Menu,URL,Price,Address
Rank,,,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,2109 W. Chicago Ave.
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9,800 W. Randolph St.
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St.
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,"914 Noyes St., Evanston"
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,825 W. Fulton Mkt.


In [31]:
df['Phone'] = Phone
df.head()

,Cafe,Menu,URL,Price,Address,Phone
Rank,,,,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,2109 W. Chicago Ave.,773-772-0406
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9,800 W. Randolph St.,312-929-4580
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St.,312-334-3688
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,"914 Noyes St., Evanston",847-475-9400
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,825 W. Fulton Mkt.,312-445-8977


In [32]:
df['Homepage'] = Homepage
df.head()

,Cafe,Menu,URL,Price,Address,Phone,Homepage
Rank,,,,,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [33]:
cols = list(df)
cols

['Cafe', 'Menu', 'URL', 'Price', 'Address', 'Phone', 'Homepage']

In [34]:
cols.index('URL')

2

In [35]:
cols[6]

'Homepage'

In [36]:
# cols.insert(6, cols.pop(cols.index('URL')))
cols.append(cols.pop(cols.index('URL')))
cols

['Cafe', 'Menu', 'Price', 'Address', 'Phone', 'Homepage', 'URL']

In [37]:
df.head()

,Cafe,Menu,URL,Price,Address,Phone,Homepage
Rank,,,,,,,
1,Old Oak Tap,BLT,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com
2,Au Cheval,Fried Bologna,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com
3,Xoco,Woodland Mushroom,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com
4,Al’s Deli,Roast Beef,http://www.chicagomag.com/Chicago-Magazine/Nov...,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net
5,Publican Quality Meats,PB&L,http://www.chicagomag.com/Chicago-Magazine/Nov...,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com


In [38]:
df = df.ix[:, cols]
df.head()

C:\Python\Anaconda3-52\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


,Cafe,Menu,Price,Address,Phone,Homepage,URL
Rank,,,,,,,
1,Old Oak Tap,BLT,$10,2109 W. Chicago Ave.,773-772-0406,theoldoaktap.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,Au Cheval,Fried Bologna,$9,800 W. Randolph St.,312-929-4580,aucheval.tumblr.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,Xoco,Woodland Mushroom,$9.50,445 N. Clark St.,312-334-3688,rickbayless.com,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,Al’s Deli,Roast Beef,$9.40,"914 Noyes St., Evanston",847-475-9400,alsdeli.net,http://www.chicagomag.com/Chicago-Magazine/Nov...
5,Publican Quality Meats,PB&L,$10,825 W. Fulton Mkt.,312-445-8977,publicanqualitymeats.com,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [39]:
df.to_csv('data/chicagomag_info2.csv', sep=',', encoding='UTF-8')

In [40]:
% ls data

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 68FA-1A40

 C:\Users\Ente_LJS\Anaconda_src\data 디렉터리

2018-11-07  오후 03:22    <DIR>          .
2018-11-07  오후 03:22    <DIR>          ..
2018-10-29  오후 04:32    <DIR>          .ipynb_checkpoints
2018-11-05  오후 03:43            11,830 2015년_범죄발생지_지역별_.csv
2018-11-07  오후 02:55             7,675 chicagomag_info.csv
2018-11-07  오후 03:22            10,404 chicagomag_info2.csv
2018-10-29  오후 04:54               310 company.csv
2018-04-09  오전 05:33             3,258 gender_submission.csv
2018-11-01  오후 04:18                62 girlgroup.json
2018-10-29  오후 04:32                21 hello.txt
2018-11-01  오후 04:41               193 hong.json
2018-10-31  오전 09:49                12 mydata.txt
2018-11-06  오전 10:48             1,140 opengov_2018_seoul_cctv_status_by_year.csv
2018-11-05  오후 03:43            12,817 public_2016_crime_st.csv
2018-11-05  오후 05:22            13,557 public_2017_seoul_cctv_st.xlsx
2018-11-05  오후 05:36               997 public_2017_seoul_cctv